In [1]:
import dash
from dash import dcc, html
from dash.dependencies import Input, Output
import pandas as pd
import plotly.express as px
import ast
import plotly.graph_objs as go
from plotly.subplots import make_subplots

# Leer los datos del archivo
df_satellite = pd.read_excel('dat_sat.xlsx')

# Convertir la cadena de la columna Dist_obj_lanza a una lista
df_satellite['Dist_obj_lanza'] = df_satellite['Dist_obj_lanza'].apply(ast.literal_eval)

# Leer los datos del archivo para el choropleth map
df_map = pd.read_excel('dat_sat_loc.xlsx')

# Obtener años y tipos de órbita
years = df_satellite['Year'].unique()
orbit_types = ['LEO', 'MEO', 'GEO']

# Crear la aplicación Dash
app_satellite = dash.Dash(__name__)

# Definir el diseño de la aplicación
app_satellite.layout = html.Div([
    html.Div([
        dcc.Graph(id='stacked-bar-chart')
    ], style={'width': '100%'}),
    html.Div([
        dcc.Graph(id='choropleth-map')
    ], style={'display': 'inline-block', 'width': '50%'}),
    html.Div([
        dcc.Graph(id='objects-distribution')
    ], style={'display': 'inline-block', 'width': '50%'})
])

@app_satellite.callback(
    [Output('stacked-bar-chart', 'figure'),
     Output('objects-distribution', 'figure'),
     Output('choropleth-map', 'figure')],
    [Input('stacked-bar-chart', 'clickData')]
)
def update_charts(selected_bar):
    # Gráfico de barras apiladas verticales
    fig_bar = px.bar(df_satellite, x='Year', y=orbit_types, barmode='stack',
                     title='Histórico de lanzamientos',
                     labels={'value': 'Número de lanzamientos', 'variable': 'Tipo de órbita'},
                     color_discrete_map={'LEO': 'lightblue', 'MEO': 'blue', 'GEO': 'darkblue'})

    # Valor predeterminado para selected_year
    selected_year = years[0]

    if selected_bar is not None:
        # Obtén el año seleccionado
        selected_year = selected_bar['points'][0]['x']

        # Filtrar datos por año seleccionado
        filtered_df = df_satellite[df_satellite['Year'] == selected_year]

        # Gráfico de distribución de objetos por lanzamiento
        fig_objects_distribution = make_subplots(rows=1, cols=1, subplot_titles=[f'Distribución de objetos por lanzamiento en {selected_year}'])

        for index, row in filtered_df.iterrows():
            objects_array = row['Dist_obj_lanza']
            fig_objects_distribution.add_trace(go.Histogram(x=objects_array, nbinsx=len(objects_array),
                                                            name=f'Lanzamiento {index + 1}',
                                                            marker_color='rgba(100, 100, 100, 0.7)'))

        fig_objects_distribution.update_layout(xaxis_title='Número de objetos por lanzamiento',
                                               yaxis_title='Frecuencia')
    else:
        # Si no se ha hecho clic en ninguna barra, muestra algo predeterminado
        fig_objects_distribution = px.line(title='Distribución de objetos por lanzamiento')

    # Choropleth map
    fig_choropleth = px.choropleth(df_map, 
                                locations='Country',
                                locationmode='country names',
                                color=df_map[str(selected_year)],  # Ajusta para usar la columna correspondiente al año seleccionado
                                hover_name='Country',
                                title=f'Paises lanzadores de objetos en {selected_year}',
                                color_continuous_scale='Greys',  # Cambia la paleta de colores a tonos azules
                                range_color=[1, df_map[str(selected_year)].max()],  # Define el rango de colores (comienza en 1)
                                )



    return fig_bar, fig_objects_distribution, fig_choropleth

# Ejecutar la aplicación
if __name__ == '__main__':
    app_satellite.run_server(debug=True)
